In [27]:
from bs4 import BeautifulSoup
import requests

walmart_url = "https://www.walmart.com/ip/SAMSUNG-Odyssey-Neo-G7-43-Mini-4K-UHD-1ms-AMD-FreeSync-Premium-Pro-Smart-Gaming-Monitor-with-HDR600-LS43CG700NNXZA/5133998070?classType=REGULAR&from=/search"
HEADERS = { 
    "accept-encoding":"gzip, deflate, br, zstd",
    "accept-language":"en-US",
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}
# response =  requests.get(walmart_url, headers=HEADERS)

# soup =  BeautifulSoup(response.text, "html.parser")

# script_tag = soup.find("script", id = "__NEXT_DATA__")
# print(script_tag)

In [71]:
from selenium import webdriver
import json 
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)
# driver = webdriver.Chrome()  # Or use another WebDriver

def get_product_links(query,page_number=1):
    search_url = f"https://www.walmart.com/search?q={query}&page={page_number}"
    driver.get(search_url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,"html.parser")

    links = soup.find_all('a',href=True)

    product_links = []

    for link in links:
        link_href = link['href']
        if "/ip/" in link_href:
            if "https" in link_href:
                full_url = link_href
            else:
                full_url = "https://walmart.com"

            product_links.append(full_url)
            
    return product_links


def extract_product_info(product_url):
    driver.get(walmart_url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    
    script_tag = soup.find("script", id="__NEXT_DATA__")
    
    data = json.loads(script_tag.string)
    initial_data = data['props']['pageProps']['initialData']['data']
    product_data = initial_data['product']
    reviews_data = initial_data.get('reviews',{})
    
    product_info = {
        "price" : product_data['priceInfo']['currentPrice']['price'],
        "review_count" : reviews_data.get("totalReviewCount",0),
        "item_id" : product_data['usItemId'],
        "avg_rating": reviews_data.get("averageOverallRating", 0),
        "product_name": product_data["name"],
        "brand": product_data.get("brand", ""),
        "availability": product_data["availabilityStatus"],
        "image_url": product_data["imageInfo"]["thumbnailUrl"],
        "short_description": product_data.get("shortDescription", "")
    }
    return product_info

def main():
    OUTPUT_FILE = "product_info.jsonl"

    with open("OUTPUTFILE" , "w") as file:
        page_number = 1
        while True:
            links = get_product_links("monitors",page_number)
            if not links or page_number > 99:
                break
            for link in links:
                try:
                    product_info = extract_product_info(link)
                    if product_info:
                        file.write(json.dumps(product_info)+"\n")
                except Exception as e:
                    print(f"Failed to process URL {link}. Error{e}")
            page_number=+1
            print(f"Search page {page_number}")
if __name__ == "__main__":
    main()

driver.quit()


Failed to process URL https://www.walmart.com/sp/track?bt=1&eventST=click&plmt=sb-search-top~desktop~&pos=2&tax=3944_1089430_1230331_1230332&rdf=1&rd=https%3A%2F%2Fwww.walmart.com%2Fip%2FSAMSUNG-27-Odyssey-G55C-QHD-165Hz-1ms-MPRT-Curved-Gaming-Monitor-LS27CG556ENXZA%2F5329046686%3FadsRedirect%3Dtrue&adUid=357c3410-b020-4c22-8ea9-01955c028a9c&mloc=sb-search-top&pltfm=desktop&pgId=monitors&pt=search&spQs=aLQNwXQGaxQ04qC1Wsxko--cx9sYYWW0MS1BwBpc9mPzmczgKSsOVgb_eO4cWcmKquhaHux2snFkN56FJheh1wUPcVmteT6AI90jQCsBapmUGSaCJdARK12S3CHNWz0oP0Zbh2XIK4pxYkwRIyCxSjNK8U9zHyfI4C3e31qbPJZSL9HZNgIWC4eGBHZdcWX816jHXR7P0zQToPh58JYLzg&storeId=945&couponState=na&bkt=ace1_default%7Cace2_default%7Cace3_default%7Cbb_ctrl-10800&/ip/SAMSUNG-27-Odyssey-G55C-QHD-165Hz-1ms-MPRT-Curved-Gaming-Monitor-LS27CG556ENXZA/5329046686. Error'NoneType' object has no attribute 'string'
Failed to process URL https://www.walmart.com/sp/track?bt=1&eventST=click&plmt=sb-search-top~desktop~&pos=4&tax=3944_1089430_1230331_1230332&rdf